In [1]:
# %load ../../../../../libraries.py
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss

pd.options.display.max_rows = 300
pd.options.display.max_columns = 100
tqdm.pandas()

from IPython.core.debugger import set_trace
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%config Completer.use_jedi = False
%reload_ext autoreload
%autoreload 2


In [2]:
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import xgboost as xgb

### Data Prep

In [3]:
# read data
df_data = pd.read_csv('../data/telecom_churn_data.csv')
df_data.shape
df_data.head().T

(99999, 226)

,0,1,2,3,4
mobile_number,7000842753,7001865778,7001625959,7001204172,7000142493
circle_id,109,109,109,109,109
loc_og_t2o_mou,0.0,0.0,0.0,0.0,0.0
std_og_t2o_mou,0.0,0.0,0.0,0.0,0.0
loc_ic_t2o_mou,0.0,0.0,0.0,0.0,0.0
last_date_of_month_6,6/30/2014,6/30/2014,6/30/2014,6/30/2014,6/30/2014
last_date_of_month_7,7/31/2014,7/31/2014,7/31/2014,7/31/2014,7/31/2014
last_date_of_month_8,8/31/2014,8/31/2014,8/31/2014,8/31/2014,8/31/2014
last_date_of_month_9,9/30/2014,9/30/2014,9/30/2014,9/30/2014,9/30/2014
arpu_6,197.385,34.047,167.69,221.338,261.636


In [4]:
# check unique counts including NAs
# df_data.nunique(dropna=False)
df_data.nunique()

mobile_number               99999
circle_id                       1
loc_og_t2o_mou                  1
std_og_t2o_mou                  1
loc_ic_t2o_mou                  1
last_date_of_month_6            1
last_date_of_month_7            1
last_date_of_month_8            1
last_date_of_month_9            1
arpu_6                      85681
arpu_7                      85308
arpu_8                      83615
arpu_9                      79937
onnet_mou_6                 24313
onnet_mou_7                 24336
onnet_mou_8                 24089
onnet_mou_9                 23565
offnet_mou_6                31140
offnet_mou_7                31023
offnet_mou_8                30908
offnet_mou_9                30077
roam_ic_mou_6                6512
roam_ic_mou_7                5230
roam_ic_mou_8                5315
roam_ic_mou_9                4827
roam_og_mou_6                8038
roam_og_mou_7                6639
roam_og_mou_8                6504
roam_og_mou_9                5882
loc_og_t2t_mou

In [5]:
# check missing values proportion
df_data.isnull().sum() / len(df_data)

mobile_number               0.000000
circle_id                   0.000000
loc_og_t2o_mou              0.010180
std_og_t2o_mou              0.010180
loc_ic_t2o_mou              0.010180
last_date_of_month_6        0.000000
last_date_of_month_7        0.006010
last_date_of_month_8        0.011000
last_date_of_month_9        0.016590
arpu_6                      0.000000
arpu_7                      0.000000
arpu_8                      0.000000
arpu_9                      0.000000
onnet_mou_6                 0.039370
onnet_mou_7                 0.038590
onnet_mou_8                 0.053781
onnet_mou_9                 0.077451
offnet_mou_6                0.039370
offnet_mou_7                0.038590
offnet_mou_8                0.053781
offnet_mou_9                0.077451
roam_ic_mou_6               0.039370
roam_ic_mou_7               0.038590
roam_ic_mou_8               0.053781
roam_ic_mou_9               0.077451
roam_og_mou_6               0.039370
roam_og_mou_7               0.038590
r

In [6]:
# handle missing values

df_data['date_of_last_rech_6'].fillna('12/31/2014', inplace=True)
df_data['date_of_last_rech_7'].fillna('12/31/2014', inplace=True)
df_data['date_of_last_rech_8'].fillna('12/31/2014', inplace=True)
df_data['date_of_last_rech_9'].fillna('12/31/2014', inplace=True)
df_data['date_of_last_rech_data_6'].fillna('12/31/2014', inplace=True)
df_data['date_of_last_rech_data_7'].fillna('12/31/2014', inplace=True)
df_data['date_of_last_rech_data_8'].fillna('12/31/2014', inplace=True)
df_data['date_of_last_rech_data_9'].fillna('12/31/2014', inplace=True)

df_data.fillna(0, inplace=True)

In [7]:
# add date diff cols
df_data['datediff_of_last_rech_6'] = pd.to_datetime(df_data['last_date_of_month_6']) - pd.to_datetime(df_data['date_of_last_rech_6'])
df_data['datediff_of_last_rech_7'] = pd.to_datetime(df_data['last_date_of_month_7']) - pd.to_datetime(df_data['date_of_last_rech_7'])
df_data['datediff_of_last_rech_8'] = pd.to_datetime(df_data['last_date_of_month_8']) - pd.to_datetime(df_data['date_of_last_rech_8'])
df_data['datediff_of_last_rech_9'] = pd.to_datetime(df_data['last_date_of_month_9']) - pd.to_datetime(df_data['date_of_last_rech_9'])
df_data['datediff_of_last_rech_data_6'] = pd.to_datetime(df_data['last_date_of_month_6']) - pd.to_datetime(df_data['date_of_last_rech_data_6'])
df_data['datediff_of_last_rech_data_7'] = pd.to_datetime(df_data['last_date_of_month_7']) - pd.to_datetime(df_data['date_of_last_rech_data_7'])
df_data['datediff_of_last_rech_data_8'] = pd.to_datetime(df_data['last_date_of_month_8']) - pd.to_datetime(df_data['date_of_last_rech_data_8'])
df_data['datediff_of_last_rech_data_9'] = pd.to_datetime(df_data['last_date_of_month_9']) - pd.to_datetime(df_data['date_of_last_rech_data_9'])

# convert to int
df_data['datediff_of_last_rech_6'] = df_data['datediff_of_last_rech_6'].apply(lambda x: x.days if x.days >= 0 else -1)
df_data['datediff_of_last_rech_7'] = df_data['datediff_of_last_rech_7'].apply(lambda x: x.days if x.days >= 0 else -1)
df_data['datediff_of_last_rech_8'] = df_data['datediff_of_last_rech_8'].apply(lambda x: x.days if x.days >= 0 else -1)
df_data['datediff_of_last_rech_9'] = df_data['datediff_of_last_rech_9'].apply(lambda x: x.days if x.days >= 0 else -1)
df_data['datediff_of_last_rech_data_6'] = df_data['datediff_of_last_rech_data_6'].apply(lambda x: x.days if x.days >= 0 else -1)
df_data['datediff_of_last_rech_data_7'] = df_data['datediff_of_last_rech_data_7'].apply(lambda x: x.days if x.days >= 0 else -1)
df_data['datediff_of_last_rech_data_8'] = df_data['datediff_of_last_rech_data_8'].apply(lambda x: x.days if x.days >= 0 else -1)
df_data['datediff_of_last_rech_data_9'] = df_data['datediff_of_last_rech_data_9'].apply(lambda x: x.days if x.days >= 0 else -1)

In [8]:
# filter high-value customers
df_data['total_rech_amt_6_7'] = df_data['total_rech_amt_6'] + \
                                df_data['total_rech_amt_7'] + \
                                df_data['total_rech_data_6'] + \
                                df_data['total_rech_data_7']

df_data['total_rech_num_6_7'] = df_data['total_rech_num_6'] + \
                                df_data['total_rech_num_7'] + \
                                df_data['count_rech_2g_6'] + \
                                df_data['count_rech_2g_7'] + \
                                df_data['count_rech_3g_6'] + \
                                df_data['count_rech_3g_7']

df_data['av_rech_amt_6_7'] = df_data['total_rech_amt_6_7'] / df_data['total_rech_num_6_7']

# calculate high-value custommer threshold
hv_thresh = df_data['av_rech_amt_6_7'].quantile(0.7)

df_hv = df_data.loc[df_data['av_rech_amt_6_7'] >= hv_thresh].copy()
df_hv.shape

(29946, 237)

In [9]:
# drop columns that don't add value
drop_cols = ['loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou', 
             'std_og_t2c_mou_6', 'std_og_t2c_mou_7', 'std_og_t2c_mou_8', 'std_og_t2c_mou_9', 
             'std_ic_t2o_mou_6', 'std_ic_t2o_mou_7', 'std_ic_t2o_mou_8', 'std_ic_t2o_mou_9']
date_cols = ['last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8', 'last_date_of_month_9', 
             'date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8', 'date_of_last_rech_9', 
             'date_of_last_rech_data_6', 'date_of_last_rech_data_7', 'date_of_last_rech_data_8', 'date_of_last_rech_data_9']
inter_cols = [col for col in df_hv.columns if ('6_7' in col) or ('av_' in col)]

drop_cols = drop_cols + date_cols + inter_cols

df_hv = df_hv.drop(drop_cols, axis=1, errors='ignore')
df_hv.shape

(29946, 207)

In [10]:
# check outliers
# df_hv.describe(percentiles=[.25,.5,.75,.90,.95,.99]).T

In [11]:
# handle outliers
# skip_cols = ['mobile_number', 'circle_id', 'total_rech_amt_6_7', 'av_rech_amt_6_7', 'total_rech_num_6_7']
# date_cols = [col for col in df_hv.columns if 'date_' in col]
# data_cols = [col for col in df_hv.columns if 'data_' in col]
# skip_cols = skip_cols + date_cols + data_cols

# for col in tqdm(df_hv.columns):
#     if col in skip_cols:
#         continue
#     df_hv = df_hv.loc[df_hv[col] <= df_hv[col].quantile(0.99)]
    
# df_hv.shape

In [12]:
# tag churn labels
df_hv['total_usage_9'] = df_hv['total_ic_mou_9'] + df_hv['total_og_mou_9'] + df_hv['vol_2g_mb_9'] + df_hv['vol_3g_mb_9']
df_hv['churn'] = df_hv['total_usage_9'].apply(lambda x: 1 if x==0 else 0)
df_hv['churn'].value_counts()

0    27284
1     2662
Name: churn, dtype: int64

In [13]:
# remove churn phase attributes (_9)
df_model = df_hv.drop([col for col in df_hv.columns if '_9' in col], axis=1, errors='ignore')
df_model.shape

(29946, 158)

### EDA

In [14]:
# fig, ax = plt.subplots(figsize=(25,25))
# sns.heatmap(df_model.corr(), ax=ax)

In [15]:
# churn rate
df_model['churn'].value_counts() / len(df_model['churn'])

0    0.911107
1    0.088893
Name: churn, dtype: float64

### Modeling - Predict Churn

In [16]:
id_cols = ['mobile_number', 'circle_id']
target_cols = ['churn']
date_cols = [col for col in df_model.columns if 'date_' in col]
feature_cols = [col for col in df_model.columns if col not in id_cols + target_cols + date_cols]

In [17]:
# train-test-split
X_train, X_test, y_train, y_test = train_test_split(df_model[feature_cols], df_model[target_cols], test_size=0.3, 
                                                    stratify=df_model[target_cols], random_state=123)

In [18]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20962, 155), (8984, 155), (20962, 1), (8984, 1))

In [19]:
y_train['churn'].value_counts()

0    19099
1     1863
Name: churn, dtype: int64

In [20]:
# resample with smote
sm = SMOTE(random_state=123)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [21]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((38198, 155), (8984, 155), (38198, 1), (8984, 1))

In [22]:
y_train['churn'].value_counts()

0    19099
1    19099
Name: churn, dtype: int64

In [23]:
y_train = y_train.values.reshape(-1)
y_test = y_test.values.reshape(-1)

In [24]:
# standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
# pca
pca = PCA(n_components=0.9, random_state=123)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
X_train.shape, X_test.shape

((38198, 58), (8984, 58))

In [26]:
# logistic regression
lr = LogisticRegression()
model_lr = lr.fit(X_train, y_train)

In [27]:
# make prediction - train
y_pred_probs = model_lr.predict_proba(X_train)[:,1]
y_pred = model_lr.predict(X_train)

In [28]:
# metrics - train
print(f'ROC AUC: {roc_auc_score(y_train, y_pred_probs)}')
print(f'Precision: {precision_score(y_train, y_pred)}')
print(f'Recall: {recall_score(y_train, y_pred)}')
print(f'F1: {f1_score(y_train, y_pred)}')

ROC AUC: 0.9177093763341646
Precision: 0.8329879613183343
Recall: 0.8839729828786848
F1: 0.857723473975665


In [29]:
# make prediction - test
y_pred_probs = model_lr.predict_proba(X_test)[:,1]
# y_pred = model_lr.predict(X_test)
y_pred = [1 if x >= 0.3 else 0 for x in y_pred_probs]

In [30]:
# metrics - test
print(f'ROC AUC: {roc_auc_score(y_test, y_pred_probs)}')
print(f'Precision: {precision_score(y_test, y_pred)}')
print(f'Recall: {recall_score(y_test, y_pred)}')
print(f'F1: {f1_score(y_test, y_pred)}')

ROC AUC: 0.883056019168738
Precision: 0.21169415292353824
Recall: 0.8836045056320401
F1: 0.34155781325592643


### Modeling - Identify Important Features

In [31]:
# train-test-split
X_train, X_test, y_train, y_test = train_test_split(df_model[feature_cols], df_model[target_cols], test_size=0.3, 
                                                    stratify=df_model[target_cols], random_state=123)

In [32]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20962, 155), (8984, 155), (20962, 1), (8984, 1))

In [33]:
y_train['churn'].value_counts()

0    19099
1     1863
Name: churn, dtype: int64

In [34]:
# resample with smote
sm = SMOTE(random_state=123)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [35]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((38198, 155), (8984, 155), (38198, 1), (8984, 1))

In [36]:
y_train['churn'].value_counts()

0    19099
1    19099
Name: churn, dtype: int64

In [37]:
y_train = y_train.values.reshape(-1)
y_test = y_test.values.reshape(-1)

In [38]:
# standardize data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [39]:
# xgboost model
model_xgb = xgb.XGBClassifier(n_jobs = -1, objective = 'binary:logistic')
model_xgb.fit(X_train, y_train)

/home/paresh/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:22:11] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [40]:
# make prediction - train
y_pred_probs = model_xgb.predict_proba(X_train)[:,1]
y_pred = model_xgb.predict(X_train)

In [41]:
# metrics - train
print(f'ROC AUC: {roc_auc_score(y_train, y_pred_probs)}')
print(f'Precision: {precision_score(y_train, y_pred)}')
print(f'Recall: {recall_score(y_train, y_pred)}')
print(f'F1: {f1_score(y_train, y_pred)}')

ROC AUC: 0.999958094348417
Precision: 0.9963940423308074
Recall: 0.9982721608461176
F1: 0.9973322173981273


In [42]:
# make prediction - test
y_pred_probs = model_xgb.predict_proba(X_test)[:,1]
# y_pred = model_xgb.predict(X_test)
y_pred = [1 if x >= 0.25 else 0 for x in y_pred_probs]

In [43]:
# metrics - test
print(f'ROC AUC: {roc_auc_score(y_test, y_pred_probs)}')
print(f'Precision: {precision_score(y_test, y_pred)}')
print(f'Recall: {recall_score(y_test, y_pred)}')
print(f'F1: {f1_score(y_test, y_pred)}')

ROC AUC: 0.9331042238962417
Precision: 0.5692742695570217
Recall: 0.7559449311639549
F1: 0.6494623655913978


In [44]:
# feature importances
pd.DataFrame(zip(df_model.columns, model_xgb.feature_importances_), 
             columns=['ColumnName', 'Importance']
            ).sort_values('Importance', ascending=False
            ).head(10)

,ColumnName,Importance
65,loc_ic_mou_6,0.264488
80,total_ic_mou_6,0.071258
14,roam_og_mou_6,0.050654
98,max_rech_amt_6,0.034374
143,fb_user_6,0.028019
108,max_rech_data_7,0.022927
142,sachet_3g_8,0.019382
101,last_day_rch_amt_6,0.016220
104,total_rech_data_6,0.012390
110,count_rech_2g_6,0.012240


### Recommendation

Based on the feature importances, we can say that minutes-of-usage for incoming calls is the most important factor in determining Churn. 

Other important factors are: 
- Outgoing minutes-of-usage while in Roaming
- Max recharge amount
- Whether customer is a Facebook services user